## Multinomial Classification
### Model Serving

In [1]:
import pandas as pd

# Features verification
from feature_engine.preprocessing import MatchVariables
# Categories verification
from feature_engine.preprocessing import MatchCategories
# Script Serving
from jpmml_evaluator import make_evaluator
from jpmml_evaluator.py4j import Py4JBackend
# REST Serving
from openscoring import Openscoring

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Data Import
df = pd.read_csv('../../../data/dry_bean_clean.csv')
# Get sample dataset for model serving
df_serve = df.sample(30)
x_serve = df_serve.drop('target', axis=1)
y_serve = df_serve[['target']]
x_serve.reset_index(drop=True, inplace=True)
y_serve.reset_index(drop=True, inplace=True)

## Features Verification

In [4]:
# Ensures that columns in serving set match those in train set
x_train = df_serve.drop('target', axis=1)
# Setup the transformer
match_cols = MatchVariables(missing_values="ignore")
match_cols.fit(x_train)
match_cols.feature_names_in_

['Area',
 'Perimeter',
 'MajorAxisLength',
 'MinorAxisLength',
 'AspectRation',
 'Eccentricity',
 'ConvexArea',
 'EquivDiameter',
 'Extent',
 'Solidity',
 'roundness',
 'Compactness',
 'ShapeFactor1',
 'ShapeFactor2',
 'ShapeFactor3',
 'ShapeFactor4']

In [5]:
# Run the transformer
x_match = match_cols.transform(x_serve)
x_match.head(5)

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,41884,748.149000,267.797808,199.374960,1.343187,0.667625,42242,230.929351,0.742703,0.991525,0.940333,0.862327,0.006394,0.002181,0.743608,0.998805
1,89064,1136.037810,431.177196,265.959802,1.621296,0.787044,90314,336.749626,0.740073,0.986164,0.867219,0.781015,0.004841,0.001111,0.609995,0.988902
2,41153,731.845052,256.623853,204.471564,1.255262,0.603614,41465,228.907950,0.782783,0.992489,0.965596,0.892072,0.006236,0.002436,0.795859,0.998682
3,180061,1615.172606,611.486741,376.471074,1.624700,0.787676,181832,478.813137,0.720286,0.990266,0.867557,0.783153,0.003396,0.000788,0.613423,0.996035
4,47775,852.110000,341.692355,178.604141,1.913127,0.852514,48428,246.635397,0.794080,0.986516,0.826836,0.721805,0.007152,0.001198,0.521003,0.996745


## Categories Verification

In [6]:
# Ensures categorical variables are of type ‘category’
mc = MatchCategories(missing_values="ignore")
# mc.fit(x_train)

In [7]:
# Run the transformer
# x_match_c = mc.transform(x_match)
# x_match_c.head(5)

## Model Script Serving

In [8]:
# Load PMML model
evaluator = make_evaluator(backend=Py4JBackend(), path="../../../data/MultinomialClassification-LogisticRegression.pmml")
# Perform automated QA
evaluator.verify()

In [9]:
# Model Serving
dfresult = evaluator.evaluateAll(x_match)
dfresult.head(5)

,target,probability(BARBUNYA),probability(BOMBAY),probability(CALI),probability(DERMASON),probability(HOROZ),probability(SEKER),probability(SIRA)
0,SEKER,1.063511e-04,4.068263e-10,5.054330e-08,8.646425e-04,1.817695e-11,9.910744e-01,7.954513e-03
1,CALI,1.551991e-03,5.552064e-03,9.928958e-01,3.897494e-20,2.887214e-08,9.611967e-13,1.327525e-07
2,SEKER,1.850116e-06,2.036167e-11,1.471680e-10,1.701867e-05,1.730132e-15,9.999288e-01,5.232816e-05
3,BOMBAY,5.826885e-07,9.999750e-01,2.439196e-05,4.858948e-47,1.358891e-23,2.647611e-20,4.250403e-25
4,HOROZ,1.605717e-04,1.713898e-10,5.476602e-04,1.445961e-05,9.620158e-01,2.203553e-09,3.726148e-02


## Model REST Serving

In [10]:
# Creating an Openscoring object
# Openscoring server have to be running on base-url
os = Openscoring(base_url = "http://localhost:8080", token="secret")

In [11]:
# Deploying a PMML model:
model_name = 'MultinomialClassification-LogisticRegression'
os.deployFile(model_name, "../../../data/MultinomialClassification-LogisticRegression.pmml")

In [13]:
# Model serving with data records from DataFrame
dfResponse = os.evaluateCsv(model_name, x_match)
print(dfResponse.head(5))

   target  probability(BARBUNYA)  probability(BOMBAY)  probability(CALI)  \
0   SEKER           1.063511e-04         4.068263e-10       5.054330e-08   
1    CALI           1.551991e-03         5.552064e-03       9.928958e-01   
2   SEKER           1.850116e-06         2.036167e-11       1.471680e-10   
3  BOMBAY           5.826885e-07         9.999750e-01       2.439196e-05   
4   HOROZ           1.605717e-04         1.713898e-10       5.476602e-04   

   probability(DERMASON)  probability(HOROZ)  probability(SEKER)  \
0           8.646425e-04        1.817695e-11        9.910744e-01   
1           3.897494e-20        2.887214e-08        9.611967e-13   
2           1.701867e-05        1.730132e-15        9.999288e-01   
3           4.858948e-47        1.358891e-23        2.647611e-20   
4           1.445961e-05        9.620158e-01        2.203553e-09   

   probability(SIRA)  
0       7.954513e-03  
1       1.327525e-07  
2       5.232816e-05  
3       4.250403e-25  
4       3.726148e-0